# Clustering For Match Making

### Remainder Problem
Instead of being left with 1 user who gets grouped with 3 bots, make the last three groups all have 3 users and 1 bot

### Clustering algorithms
Split based on ranked squad_score: Simpliest mvp
- ended up implementing this method due to time constrants. The other methods involve utilizing all of the metrics which invloved setting up a DS database which we dodn't have implemented at this time.

K Means: Doesn't split into even groups like we need, cluster into 5 similar tiers and then group within each cluster based off of Squad Score.

KNN: Select a random user, grab the 3 closest neighbors, repeat until everyone is matched

Agglomarative clustering: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html

linear sum assignment: https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.linear_sum_assignment.html

In [2]:
# Imports
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

In [3]:
# Read in metrics csv
df = pd.read_csv('../data/squad_score_metrics.csv', index_col='Unnamed: 0')
df.head()

,story_id,story_length,avg_word_len,quotes_num,unique_words_num,adj_num,squad_score
0,3132,1375,5.092593,6,138,15,47.843668
1,3104,903,4.961538,0,110,12,32.839743
2,3103,750,5.000000,1,93,10,29.830446
3,3117,439,4.877778,1,56,3,17.120997
4,3102,1812,4.897297,0,193,22,54.416686


### Using K Means Clustering
- Segment the users into different clusters based on all of the metrics - 5 'tiers'
- Within their clusters, group into groups of 4 based on their Squad Score

In [4]:
# Get clusters

# Instantiate scaler
scaler = StandardScaler()

# Pull out features
features = df.drop('story_id', axis=1)

# Scale data
norm_x = scaler.fit_transform(features)

# Instantiate model - 5 clusters
model= KMeans(n_clusters = 5)

# Predict clusters
df['cluster'] = model.fit_predict(norm_x)

# View df
df.head()

,story_id,story_length,avg_word_len,quotes_num,unique_words_num,adj_num,squad_score,cluster
0,3132,1375,5.092593,6,138,15,47.843668,0
1,3104,903,4.961538,0,110,12,32.839743,2
2,3103,750,5.000000,1,93,10,29.830446,2
3,3117,439,4.877778,1,56,3,17.120997,2
4,3102,1812,4.897297,0,193,22,54.416686,0


In [5]:
# Cluster distribution
df['cluster'].value_counts()

0    44
2    44
3    39
1    26
4    14
Name: cluster, dtype: int64

The associated number of the clusters change each time you run the model, but the distribution remains the exact same

# MVP Clustering from Squad Scores

Grouping function:
- uses the remainder from len(df) % 4 to make decisions of how to group the last users
- edge case conditions handles if there are less than 6 users, otherwise it would return some keys with blank values

In [6]:
def group_4(df):
    '''
    Function to split given dataframe into groups of 4 based on their ranked squad_scores
    When there is a remainder of users not evenly divisable by 4, it will split the remainder so there is never more than 1 computer user in a group, unless there are less that 3 users.

        Input: df to be grouped containing the column 'squad_score' and 'story_id'
        Output: Dictionary of groupings. {group #: list of story_id's}

    '''
    # Rank by squad_score
    df = df.sort_values(by= ['squad_score'], ascending= False)

    # Initial variables
    split = len(df) // 4
    remainder = len(df) % 4
    group_dict = {}

    # Edge Cases: 
    # - less than 4, they are all in one group
    # - 5, one group of 3 one group of 2
    if len(df) == 5:
        group_dict[1] = list(df['story_id'][:3])
        group_dict[2] = list(df['story_id'][3:])
        return group_dict
    
    if len(df) < 4:
        group_dict[1] = list(df['story_id'][:])
        return group_dict

    # If the remainder is 3 -> last group will be a group of 3 users
    if remainder == 3:
        for i in range(split):
            # Group by top 4 squad scores
            group_dict[i+1] = list(df['story_id'][:4])
            # Drop stories you have grouped already 
            df = df[4:]
        
        # Final group is the last 3 remainders
        group_dict[split +1] = list(df['story_id'][:])
        return group_dict

    # If the remainder is 2 -> last 2 groups will be groups of 3
    elif remainder == 2:
        # Leave the last 2 groups to split into 2 groups of 3
        for i in range(split -1):
            # Group by top 4 squad scores
            group_dict[i+1] = list(df['story_id'][:4])
            # Drop stories you have grouped already
            df = df[4:]

        # The last two groups will be groups of 3
        group_dict[split] = list(df['story_id'][:3])
        group_dict[split + 1] = list(df['story_id'][3:])
        return group_dict

    # If the remainder is 1 -> last 3 groups will be groups of 3
    elif remainder == 1:
        # Leave the last 3 groups to be split into 3 groups of 3
        for i in range(split -2):
            # Group by top 4 squad scores
            group_dict[i+1] = list(df['story_id'][:4])
            # Drop stories you have already grouped
            df = df[4:]

        # The last three groups as groups of 3
        group_dict[split -1] = list(df['story_id'][:3])
        group_dict[split] = list(df['story_id'][3:6])
        group_dict[split + 1] = list(df['story_id'][6:])
        return group_dict
    
    # If the remainder is 0, split evenly by 4
    elif remainder == 0:
        for i in range(split):
            # Group by top 4 squad scores
            group_dict[i+1] = list(df['story_id'][:4])
            # Drop stories you have already grouped
            df = df[4:]
        return group_dict

    else:
        return 'Invalid number of remaining users'

In [7]:
# Extract each cluster 'tier'
first = df[df['cluster']== 0]
second = df[df['cluster']== 1]
third = df[df['cluster']== 2]
fourth = df[df['cluster']== 3]
fifth = df[df['cluster']== 4]

In [8]:
# Create all the cluster dictionaries
first_cluster = group_4(first)
second_cluster = group_4(second)
third_cluster = group_4(third)
fourth_cluster = group_4(fourth)
fifth_cluster = group_4(fifth)

In [9]:
# View each cluster and their groupings
print(f'First Cluster: {first_cluster}')
print(f'Second Cluster: {second_cluster}')
print(f'Third Cluster: {third_cluster}')
print(f'Fourth Cluster: {fourth_cluster}')
print(f'Fifth Cluster: {fifth_cluster}')

First Cluster: {1: [5105, 5217, 3115, 5251], 2: [5113, 5237, 5125, 3102], 3: [5218, 5255, 3219, 5204], 4: [5121, 5202, 5259, 3204], 5: [5263, 5210, 5203, 3122], 6: [3109, 5246, 3213, 3239], 7: [5114, 3132, 3224, 5108], 8: [3101, 5228, 3114, 3211], 9: [3217, 5225, 3220, 5249], 10: [5239, 3232, 5260, 3216], 11: [5240, 5102, 5229, 3247]}
Second Cluster: {1: [5219, 5117, 5119, 5107], 2: [5118, 5222, 5215, 5227], 3: [5124, 5106, 5129, 5224], 4: [3130, 3209, 5247, 5252], 5: [5206, 3230, 5248, 5131], 6: [5262, 5111, 5253], 7: [5208, 5257, 5261]}
Third Cluster: {1: [5209, 5221, 3118, 5243], 2: [3215, 3210, 5123, 5232], 3: [3227, 3128, 3223, 3123], 4: [5120, 3116, 5207, 3104], 5: [5258, 3111, 3225, 5214], 6: [3226, 3103, 3238, 3206], 7: [3207, 3125, 3113, 3107], 8: [3222, 5110, 3124, 3235], 9: [3131, 3119, 3236, 5233], 10: [3121, 3214, 3110, 3127], 11: [3202, 3117, 3240, 3229]}
Fourth Cluster: {1: [5122, 5230, 5244, 3221], 2: [3234, 5132, 3246, 3129], 3: [5115, 5126, 3245, 5238], 4: [5112, 3243

In [10]:
# Testing edge cases
remainder_0 = first[3:]
remainder_1 = first[2:]
remainder_2 = first[1:]
remainder_3 = first
small_num_9 = first[-9:]
small_num_5 = first[-5:]
small_num_4 = first[-4:]
small_num_3 = first[-3:]
small_num_2 = first[-2:]
small_num_1 = first[-1:]

In [11]:
clust_dict = group_4(small_num_1)
clust_dict

{1: [5102]}

In [12]:
# Sanity check - each story from the cluster was grouped
unique = set()

for inner_list in clust_dict.values():
    for item in inner_list:
        unique.add(item)
    
len(unique) == len(first)

False

## KNN
- Pull a random story, find the three nearest stories to create a group
- Drop the stories that you have already grouped
- TODO: Deal with the remainder problem

In [17]:
# Wrap in function that continues until the remainder problem
# Refit the data after you drop the grouped stories 
    # - otherwise it could suggest a story that we have already grouped and dropped

def group_nn(df):
    '''
    Function creates groups of four from the input df by using Nearest Neighbors
    Pulls one user, finds the three most similar users in the cohort to form the group
    When there is a remainder of users not evenly divisable by 4, it will duplicate and entries, and have a couple of players play twice

        Input: df to be grouped with the ID column first with ID being Integers
        Output: dictionary of groupings {group ID #: list of story_id's}
    '''
    # Empty dictionary to store the groupings
    groups = {}

    # Instantiate scaler
    scaler = StandardScaler()

    # Pull out features
    features = df.drop(df.columns[0], axis=1)

    # Scale data
    norm_x = scaler.fit_transform(features)

    # Turn into df
    df_norm_x = pd.DataFrame(norm_x)

    # Instantiate model - groups of 4
    nn = NearestNeighbors(n_neighbors=4, algorithm='kd_tree')

    # Counter to use as key for groups in dictionary
    counter = 1

    # Grab a copy of the df before taking it apart to deal with the remainder problem
    df_copy = df

    # While loop that takes the top user and creates a group with the its three closest users
    # Drops grouped users and continues until there are less than 12 users left to group
    # Remainder problem will be dealt with after the while loop runs
    while len(df_norm_x) >4:
        # Fit the nearest neighbors model
        nn.fit(df_norm_x)

        # Find nearest neighbors
        array_1, array_2 = nn.kneighbors([df_norm_x.iloc[0].values])

        # Put story_id list into groups dictionary
        groups[counter] = [df[df.columns[0]][item] for item in array_2[0]]

        # Increment the counter
        counter += 1

        # Drop the users you have already grouped
        # From both df's that you are using
        df_norm_x = df_norm_x.drop(array_2[0])
        df = df.drop(array_2[0])

        # Reset the index
        # For both datasets that you are using
        df_norm_x.reset_index(inplace= True, drop= True)
        df.reset_index(inplace= True, drop= True)

    # Drop the remainders from copy of df to find most similar
    for i in range(len(df)):
        df_copy = df_copy[df_copy[df_copy.columns[0]] != int(df.iloc[i][0])]

    # Do preproccesing done above
    df_copy.reset_index(inplace=True, drop=True)
    features_copy = df_copy.drop(df.columns[0], axis=1)
    norm_x_copy = scaler.fit_transform(features_copy)
    df_norm_x_copy = pd.DataFrame(norm_x_copy)
    
    # Fit to KNN model
    nn.fit(df_norm_x_copy)
    
    # TODO: Finish remainder problem, going to get it working with perfect numbers first

    return groups

In [18]:
# Group 167 stories from db
nn_groups = group_nn(df)
nn_groups

{1: [3132, 3220, 3114, 5260],
 2: [3104, 3103, 3223, 3116],
 3: [3117, 3127, 3214, 3110],
 4: [3102, 5263, 5125, 5113],
 5: [3105, 5104, 5109, 5116],
 6: [3129, 3243, 5112, 3234],
 7: [3111, 5214, 3207, 3124],
 8: [3118, 3215, 5123, 3128],
 9: [3120, 3231, 3201, 3244],
 10: [3121, 3202, 5233, 3107],
 11: [3119, 3131, 3206, 3238],
 12: [3126, 5101, 3241, 3208],
 13: [3109, 5218, 3122, 5203],
 14: [3106, 3228, 3205, 3108],
 15: [3101, 5228, 3211, 3224],
 16: [3130, 5106, 5224, 5227],
 17: [3112, 5238, 5216, 5126],
 18: [3115, 5251, 5237, 5114],
 19: [3123, 5258, 5207, 3225],
 20: [3125, 5120, 3236, 3222],
 21: [3113, 3235, 5232, 3210],
 22: [3245, 3218, 5264, 5132],
 23: [3216, 5102, 5239, 5240],
 24: [3229, 3226, 3240, 5110],
 25: [3227, 5209, 5243, 5221],
 26: [3221, 5115, 3246, 3237],
 27: [3219, 3239, 5217, 5105],
 28: [3217, 3213, 5210, 3247],
 29: [3232, 5225, 5249, 5121],
 30: [3204, 5246, 5255, 5202],
 31: [3203, 5256, 5103, 3248],
 32: [3212, 5242, 5245, 5220],
 33: [3209, 3230,

In [16]:
df.head()

,story_id,story_length,avg_word_len,quotes_num,unique_words_num,adj_num,squad_score,cluster
0,3132,1375,5.092593,6,138,15,47.843668,0
1,3104,903,4.961538,0,110,12,32.839743,2
2,3103,750,5.000000,1,93,10,29.830446,2
3,3117,439,4.877778,1,56,3,17.120997,2
4,3102,1812,4.897297,0,193,22,54.416686,0


In [15]:
# Sanity check - each story from the cluster was grouped
unique = set()

for inner_list in nn_groups.values():
    for item in inner_list:
        unique.add(item)
    
len(unique) == len(df) - 3

True